In [ ]:
import random
import numpy as np

# Define classes


class Student:
    def __init__(self, id, size):
        self.id = id
        self.size = size


class Course:
    def __init__(self, id, professor, student_group, time):
        self.id = id
        self.professor = professor
        self.student = student_group
        self.time = time


class Room:
    def __init__(self, id, size):
        self.id = id
        self.size = size


class Slot:
    def __init__(self, id):
        self.id = id
        self.day = id[:3]
        self.position = int(id[3])


# Data
students = {
    "SEM101": Student("SEM101", 600), "SEM102": Student("SEM102", 550), "SEM103": Student("SEM103", 500)
}

courses = {
    "CS101": Course("CS101", "PROF001", "SEM101", 3), "CS102": Course("CS102", "PROF002", "SEM102", 3),
    "CS103": Course("CS103", "PROF003", "SEM103", 3), "CS104": Course("CS104", "PROF001", "SEM101", 3),
    "CS105": Course("CS105", "PROF002", "SEM102", 3), "CS106": Course("CS106", "PROF003", "SEM103", 3),
    "CS107": Course("CS107", "PROF004", "SEM101", 3), "CS108": Course("CS108", "PROF005", "SEM102", 3),
    "CS109": Course("CS109", "PROF001", "SEM103", 3), "CS110": Course("CS110", "PROF002", "SEM101", 3),
    "CS111": Course("CS111", "PROF003", "SEM102", 3), "CS112": Course("CS112", "PROF004", "SEM103", 3),
    "CS113": Course("CS113", "PROF005", "SEM101", 3), "CS114": Course("CS114", "PROF001", "SEM102", 3),
    "CS115": Course("CS115", "PROF002", "SEM103", 3), "CS116": Course("CS116", "PROF002", "SEM101", 2)
}

rooms = {
    "ROOM001": Room("ROOM001", 600), "ROOM002": Room("ROOM002", 550), "ROOM003": Room("ROOM003", 500)
}

slots = [
    Slot("MON1"), Slot("MON2"), Slot("MON3"), Slot("MON4"),
    Slot("TUE1"), Slot("TUE2"), Slot("TUE3"), Slot("TUE4"),
    Slot("WED1"), Slot("WED2"), Slot("WED3"), Slot("WED4"),
    Slot("THU1"), Slot("THU2"), Slot("THU3"), Slot("THU4"),
    Slot("FRI1"), Slot("FRI2"), Slot("FRI3"), Slot("FRI4")
]

# Fitness evaluation function


def fitness(solution):
    room_overbooking = 0
    slot_conflicts = 0
    prof_conflicts = 0

    slot_assignments = {}
    prof_assignments = {}

    # Evaluate room overbooking and slot conflicts
    for entry in solution:
        course_id, student_group, room_id, slot_id, professor = entry
        course = courses[course_id]
        room = rooms[room_id]

        # Room overbooking
        if room.size < students[student_group].size:
            room_overbooking += (students[student_group].size - room.size)

        # Slot conflicts (same student in multiple classes at the same time)
        if slot_id not in slot_assignments:
            slot_assignments[slot_id] = []
        slot_assignments[slot_id].append((student_group, course_id))

        # Professor conflicts (same professor teaching multiple courses at the same time)
        if course.professor not in prof_assignments:
            prof_assignments[course.professor] = []
        prof_assignments[course.professor].append(slot_id)

    # Calculate slot conflicts
    for slot_id, assignments in slot_assignments.items():
        student_groups_in_slot = {}
        for student_group, course_id in assignments:
            if student_group in student_groups_in_slot:
                slot_conflicts += 1  # Conflict when a student is assigned to two courses at the same time
            else:
                student_groups_in_slot[student_group] = course_id

    # Calculate professor conflicts
    for professor, assigned_slots in prof_assignments.items():
        if len(assigned_slots) > len(set(assigned_slots)):  # Duplicate slots means conflict
            prof_conflicts += 1

    return room_overbooking, slot_conflicts, prof_conflicts


# Updated create_population function
def create_population(pop_size):
    population = []
    for _ in range(pop_size):
        solution = []
        room_slot_assignments = {room.id: set()
                                 # Track slots per room
                                 for room in rooms.values()}
        prof_slot_assignments = {prof: set() for prof in set(
            course.professor for course in courses.values())}  # Track professor slots

        for course_id in courses:
            course = courses[course_id]
            student_group = course.student  # Get student group
            selected_room = random.choice(list(rooms.values()))  # Random room

            # Find consecutive slots for the course
            valid_slots = [
                slot for slot in slots if slot not in room_slot_assignments[selected_room.id]]

            for i in range(len(valid_slots) - (course.time - 1)):
                # Check if the slots are consecutive within the same day
                valid_consecutive = True
                for j in range(course.time - 1):
                    current_slot = valid_slots[i + j]
                    next_slot = valid_slots[i + j + 1]

                    # Check if they are not on the same day or if they are not consecutive in time
                    if current_slot.day != next_slot.day or next_slot.position != current_slot.position + 1:
                        valid_consecutive = False
                        break

                # If all slots are consecutive and free, assign them
                if valid_consecutive and all(valid_slots[i + j] not in room_slot_assignments[selected_room.id] and valid_slots[i + j] not in prof_slot_assignments[course.professor] for j in range(course.time)):
                    assigned_slots = [valid_slots[i + j]
                                      for j in range(course.time)]

                    # Assign slots to the selected room
                    room_slot_assignments[selected_room.id].update(
                        assigned_slots)
                    prof_slot_assignments[course.professor].update(
                        assigned_slots)

                    # Add to the solution
                    for slot in assigned_slots:
                        solution.append(
                            (course_id, student_group, selected_room.id, slot.id, course.professor))

                    break

        population.append(solution)
    return population


# NSGA-II selection based on non-dominated sorting and crowding distance
def non_dominated_sorting(population, fitness_values):
    fronts = [[]]
    rank = [0] * len(population)
    domination_count = [0] * len(population)
    dominated_solutions = [[] for _ in range(len(population))]

    for p in range(len(population)):
        for q in range(len(population)):
            if dominates(fitness_values[p], fitness_values[q]):
                dominated_solutions[p].append(q)
            elif dominates(fitness_values[q], fitness_values[p]):
                domination_count[p] += 1

        if domination_count[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    front_counter = 0
    while len(fronts[front_counter]) > 0:
        next_front = []
        for p in fronts[front_counter]:
            for q in dominated_solutions[p]:
                domination_count[q] -= 1
                if domination_count[q] == 0:
                    rank[q] = front_counter + 1
                    next_front.append(q)
        front_counter += 1
        fronts.append(next_front)

    return fronts


# Crowding distance for maintaining diversity in the population
def crowding_distance(front, fitness_values):
    distance = [0] * len(front)
    if len(front) > 1:  # Only calculate crowding distance if there are at least two individuals in the front
        sorted_by_first = sorted(front, key=lambda x: fitness_values[x][0])
        sorted_by_second = sorted(front, key=lambda x: fitness_values[x][1])

        # Set infinity for boundary individuals
        distance[0] = distance[-1] = float('inf')
        for i in range(1, len(front) - 1):
            distance[i] = (fitness_values[sorted_by_first[i + 1]][0] - fitness_values[sorted_by_first[i - 1]][0]) + \
                          (fitness_values[sorted_by_second[i + 1]][1] -
                           fitness_values[sorted_by_second[i - 1]][1])

    return distance


# Crossover function
def crossover(parent1, parent2):

    def get_crosses(parent):
        parent_crosses = set()
        selected_parent = set()
        for index, course in enumerate(parent):
            if course[0] not in selected_parent:
                parent_crosses.add(index)
        return parent_crosses

    cross_points = get_crosses(parent1).intersection(get_crosses(parent2))
    crossover_point = random.choice(list(cross_points))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


# Mutation function
# def mutate(solution):
#     mutation_point = random.randint(0, len(solution) - 1)
#     course_id, student_group, _, _, _ = solution[mutation_point]

#     new_room = random.choice(list(rooms.values())).id  # Random room
#     new_slot = random.choice(slots).id  # Random slot

#     professor = courses[course_id].professor
#     # Update with new room and slot, keep course_id and student_group the same
#     solution[mutation_point] = (course_id, student_group, new_room, new_slot, professor)
#     return solution


# Tournament selection for parents
def tournament_selection(population, fitness_values):
    selected_parents = random.sample(list(range(len(population))), 2)
    p1, p2 = selected_parents[0], selected_parents[1]

    if dominates(fitness_values[p1], fitness_values[p2]):
        return p1
    else:
        return p2


# Domination check
def dominates(fit1, fit2):
    return (fit1[0] <= fit2[0] and fit1[1] <= fit2[1] and fit1[2] <= fit2[2]) and \
           (fit1[0] < fit2[0] or fit1[1] < fit2[1] or fit1[2] < fit2[2])


# Run NSGA-II algorithm
def run_nsga2(pop_size=50, generations=100):
    population = create_population(pop_size)
    for generation in range(generations):
        # Evaluate the fitness of the current population
        fitness_values = [fitness(solution) for solution in population]

        # Non-dominated sorting
        fronts = non_dominated_sorting(population, fitness_values)
        new_population = []

        # Iterate through each front and apply crossover and mutation
        for front in fronts:
            distances = crowding_distance(front, fitness_values)
            sorted_front = sorted(
                front, key=lambda x: distances[front.index(x)], reverse=True)

            # Select parents from the sorted front
            while len(new_population) < pop_size and len(sorted_front) > 1:
                parent1 = population[sorted_front[0]]
                parent2 = population[sorted_front[1]]

                # Crossover
                child1, child2 = crossover(parent1, parent2)

                # # Mutation
                # child1 = mutate(child1)
                # child2 = mutate(child2)

                # Add children to the new population
                new_population.append(child1)
                new_population.append(child2)

        # Truncate to maintain population size
        population = new_population[:pop_size]

        # Print the progress
        if generation % 10 == 0:
            print(
                f"Generation {generation}: Population size {len(population)}")

    return population


# Running the algorithm
best_solution = run_nsga2()
print(f"Best solution: {best_solution}")

Generation 0: Population size 50
Generation 10: Population size 50
Generation 20: Population size 50
Generation 30: Population size 50
Generation 40: Population size 50
Generation 50: Population size 50
Generation 60: Population size 50
Generation 70: Population size 50
Generation 80: Population size 50
Generation 90: Population size 50
Best solution: [[('CS101', 'SEM101', 'ROOM001', 'MON1', 'PROF001'), ('CS101', 'SEM101', 'ROOM001', 'MON2', 'PROF001'), ('CS101', 'SEM101', 'ROOM001', 'MON3', 'PROF001'), ('CS102', 'SEM102', 'ROOM001', 'TUE1', 'PROF002'), ('CS102', 'SEM102', 'ROOM001', 'TUE2', 'PROF002'), ('CS102', 'SEM102', 'ROOM001', 'TUE3', 'PROF002'), ('CS103', 'SEM103', 'ROOM003', 'MON1', 'PROF003'), ('CS103', 'SEM103', 'ROOM002', 'MON2', 'PROF003'), ('CS103', 'SEM103', 'ROOM002', 'MON3', 'PROF003'), ('CS104', 'SEM101', 'ROOM001', 'WED1', 'PROF001'), ('CS104', 'SEM101', 'ROOM001', 'WED2', 'PROF001'), ('CS104', 'SEM101', 'ROOM001', 'WED3', 'PROF001'), ('CS105', 'SEM102', 'ROOM002', '

In [ ]:
import csv


def save_solution_to_csv(solution, filename="timetable2.csv"):
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Course ID", "Student Group", "Room ID", "Slot ID"])
        for entry in solution:
            writer.writerow(entry)


# Running NSGA-II to generate the timetable
final_population = run_nsga2()

# Get the best solution (assuming first is the best)
best_solution = final_population[0]

# Save the best solution to CSV
save_solution_to_csv(final_population[0], "timetable8.csv")

print("Timetable saved to timetable.csv")

Generation 0: Population size 50
Generation 10: Population size 50
Generation 20: Population size 50
Generation 30: Population size 50
Generation 40: Population size 50
Generation 50: Population size 50
Generation 60: Population size 50
Generation 70: Population size 50
Generation 80: Population size 50
Generation 90: Population size 50
Timetable saved to timetable.csv


: 

In [ ]:
# # { Slot: MON1
#        ROOM : H001
#    Activity: {
#    code : IT001
#   Teacher_id : LEC001
# Student_groups : [
#  S001, S002
# ]
# }
# }


#     #   {
#     #     "code": "AC-001",
#     #     "subject": "IT1010",
#     #     "teacher_ids": "FA0000004"
#     #     "subgroup_ids": [
#     #       "Y1S1.1",
#     #       "Y1S1.2",
#     #       "Y1S1.3",
#     #       "Y1S1.4",
#     #       "Y1S1.5"
#     #     ],
#     #     "duration": 2,
#     #     "slots": [
#     #       MON1,
#     #       MON2
#     #      ]
#     #     "room" : [
#     #      "H001",
#     #      "H002"
#     #      ]
#     #   },


# class Period:
#     def __init__(self, *args):
#         self.slot = args[0]
#         self.space = args[1]
#         self.activity = args[2]